In [4]:
import os
import stata_setup
## 设置pystata目录
stata_setup.config(os.getenv("STATA_SYSDIR"), 'mp')


  ___  ____  ____  ____  ____ ©
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Single-user 2-core  perpetual
Serial number: 501806366048
  Licensed to: wxg
               wxg

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000; see help set_maxvar.


In [8]:
from pystata import stata

stata.run('use womenwk.dta, replace')

pd_data = stata.pdataframe_from_data()
pd_data.head()

,c1,c2,u,v,county,age,education,married,children,select,wagefull,wage,lw,work,lwf
0,-0.436205,-0.096918,-0.218103,-0.375727,1.0,22,10,1,0,16.791273,12.782769,8.988466e+307,8.988466e+307,0.0,0.000000
1,0.352141,0.300476,0.176070,0.461234,2.0,36,10,1,0,32.434814,20.312845,2.031285e+01,3.011253e+00,1.0,3.011253
2,1.077247,-1.595963,0.538623,-0.376244,3.0,28,10,1,0,19.185072,23.063478,8.988466e+307,8.988466e+307,0.0,0.000000
3,1.021283,-1.710498,0.510642,-0.496999,4.0,37,10,1,0,21.336010,24.527700,8.988466e+307,8.988466e+307,0.0,0.000000
4,-0.442960,0.308340,-0.221480,-0.092511,5.0,39,10,1,1,31.989870,16.142241,1.614224e+01,2.781440e+00,1.0,2.781440


作为对照，首先使用$OLS$进行线性概率模型（$PLM$）估计

In [10]:
stata.run('reg work age married children education, r')


Linear regression                               Number of obs     =      2,000
                                                F(4, 1995)        =     192.58
                                                Prob > F          =     0.0000
                                                R-squared         =     0.2026
                                                Root MSE          =     .41992

------------------------------------------------------------------------------
             |               Robust
        work | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
         age |   .0102552   .0012236     8.38   0.000     .0078556    .0126548
     married |   .1111116   .0226719     4.90   0.000     .0666485    .1555748
    children |   .1153084   .0056978    20.24   0.000     .1041342    .1264827
   education |   .0186011   .0033006     5.64   0.000     .0121282     .025074
       _cons |

其次，使用$Logit$进行估计

In [11]:
stata.run('logit work age married children education, nolog')


Logistic regression                                     Number of obs =  2,000
                                                        LR chi2(4)    = 476.62
                                                        Prob > chi2   = 0.0000
Log likelihood = -1027.9144                             Pseudo R2     = 0.1882

------------------------------------------------------------------------------
        work | Coefficient  Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
         age |   .0579303    .007221     8.02   0.000     .0437773    .0720833
     married |   .7417775   .1264705     5.87   0.000     .4938998    .9896552
    children |   .7644882   .0515289    14.84   0.000     .6634935     .865483
   education |   .0982513   .0186522     5.27   0.000     .0616936     .134809
       _cons |  -4.159247   .3320401   -12.53   0.000    -4.810034   -3.508461
--------------------------------------------------

上表显示，准$R^2$为0.19。$LR$统计量为476，62，对应的$p$值为0.00， 故整个方程所有系数（除常数项外）的联合显著性很高。下面，使用稳健标准误进行$logit$估计

In [12]:
stata.run('logit work age married children education, r nolog')


Logistic regression                                     Number of obs =  2,000
                                                        Wald chi2(4)  = 344.54
                                                        Prob > chi2   = 0.0000
Log pseudolikelihood = -1027.9144                       Pseudo R2     = 0.1882

------------------------------------------------------------------------------
             |               Robust
        work | Coefficient  std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
         age |   .0579303   .0072054     8.04   0.000     .0438079    .0720527
     married |   .7417775   .1272191     5.83   0.000     .4924326    .9911224
    children |   .7644882   .0497584    15.36   0.000     .6669635    .8620129
   education |   .0982513    .019011     5.17   0.000     .0609904    .1355121
       _cons |  -4.159247    .327398   -12.70   0.000    -4.800936   -3.517559
--------------

对比以上两个表格可知，稳健标准误与普通标准误非常接近，故大致可以不必担心模型设定问题。由于各解释变量（$age$, $married$, $children$, $education$）的最小变化量至少为1单位，为了便于解释回归结果，下面让$Stata$汇报几率比而非系数

In [13]:
stata.run('logit work age married children education, or nolog')


Logistic regression                                     Number of obs =  2,000
                                                        LR chi2(4)    = 476.62
                                                        Prob > chi2   = 0.0000
Log likelihood = -1027.9144                             Pseudo R2     = 0.1882

------------------------------------------------------------------------------
        work | Odds ratio   Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
         age |   1.059641   .0076517     8.02   0.000      1.04475    1.074745
     married |   2.099664   .2655457     5.87   0.000     1.638694    2.690307
    children |   2.147895   .1106786    14.84   0.000     1.941563    2.376153
   education |    1.10324   .0205779     5.27   0.000     1.063636    1.144318
       _cons |   .0156193   .0051862   -12.53   0.000     .0081476     .029943
--------------------------------------------------

上表显示，在给定其他变量的情况下，已婚妇女参加工作的几率比是未婚妇女的2.10倍（即高出$110%$）;而年龄增加$1$岁，参加工作的几率比就会增加$6\%$；其他变量对应的几率比可以类似解释。为了与$OLS$估计的回归系数比较，计算$Probit$模型的平均边际效应

In [14]:
stata.run('margins, dydx(*)')


Average marginal effects                                 Number of obs = 2,000
Model VCE: OIM

Expression: Pr(work), predict()
dy/dx wrt:  age married children education

------------------------------------------------------------------------------
             |            Delta-method
             |      dy/dx   std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
         age |   .0099674   .0011682     8.53   0.000     .0076778    .0122569
     married |    .127629    .021152     6.03   0.000     .0861717    .1690862
    children |   .1315365    .007073    18.60   0.000     .1176736    .1453994
   education |   .0169049   .0031243     5.41   0.000     .0107814    .0230285
------------------------------------------------------------------------------


简单目测可知，$Logit$模型的平均边际效应与$OLS$回归系数相差不大。为了演示的目的，下面计算在样本均值处的边际效应

In [15]:
stata.run('margins, dydx(*) atmeans')


Conditional marginal effects                             Number of obs = 2,000
Model VCE: OIM

Expression: Pr(work), predict()
dy/dx wrt:  age married children education
At: age       = 36.208 (mean)
    married   =  .6705 (mean)
    children  = 1.6445 (mean)
    education = 13.084 (mean)

------------------------------------------------------------------------------
             |            Delta-method
             |      dy/dx   std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
         age |   .0115031   .0014236     8.08   0.000     .0087129    .0142934
     married |   .1472934   .0248209     5.93   0.000     .0986453    .1959415
    children |    .151803   .0093768    16.19   0.000     .1334249    .1701812
   education |   .0195096   .0036991     5.27   0.000     .0122596    .0267596
------------------------------------------------------------------------------


对比以上两个表格的输出结果可知，在样本均值处的边际效应与平均边际效应应有所不同。如果只关心变量$age$在"age=30"处的边际效应，可输入命令

In [16]:
stata.run('margins, dydx(age) at(age=30)')


Average marginal effects                                 Number of obs = 2,000
Model VCE: OIM

Expression: Pr(work), predict()
dy/dx wrt:  age
At: age = 30

------------------------------------------------------------------------------
             |            Delta-method
             |      dy/dx   std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
         age |    .011179   .0014719     7.59   0.000      .008294    .0140639
------------------------------------------------------------------------------


从上表可知，对比此非线性模型，变量$age$在"age=30"处的边际效应并不等于该变量的平均边际效应，也不等于在样本均值处的边际效应。下面计算$Logit$模型准确预测的比率

In [17]:
stata.run('estat clas')


Logistic model for work

              -------- True --------
Classified |         D            ~D  |      Total
-----------+--------------------------+-----------
     +     |      1177           361  |       1538
     -     |       166           296  |        462
-----------+--------------------------+-----------
   Total   |      1343           657  |       2000

Classified + if predicted Pr(D) >= .5
True D defined as work != 0
--------------------------------------------------
Sensitivity                     Pr( +| D)   87.64%
Specificity                     Pr( -|~D)   45.05%
Positive predictive value       Pr( D| +)   76.53%
Negative predictive value       Pr(~D| -)   64.07%
--------------------------------------------------
False + rate for true ~D        Pr( +|~D)   54.95%
False - rate for true D         Pr( -| D)   12.36%
False + rate for classified +   Pr(~D| +)   23.47%
False - rate for classified -   Pr( D| -)   35.93%
--------------------------------------------------
Cor

上表显示，正确预测的比率为$(1177+296)/2000=73.65\%$。为了演示的目的，假设年龄相同的个体存在组内相关，故使用$age$为聚类变量来计算聚类稳健的标准误

In [18]:
stata.run('logit work age married children education, nolog vce(cluster age)')


Logistic regression                                     Number of obs =  2,000
                                                        Wald chi2(4)  = 576.81
                                                        Prob > chi2   = 0.0000
Log pseudolikelihood = -1027.9144                       Pseudo R2     = 0.1882

                                   (Std. err. adjusted for 40 clusters in age)
------------------------------------------------------------------------------
             |               Robust
        work | Coefficient  std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
         age |   .0579303   .0055907    10.36   0.000     .0469728    .0688879
     married |   .7417775   .1084937     6.84   0.000     .5291337    .9544213
    children |   .7644882   .0540759    14.14   0.000     .6585014     .870475
   education |   .0982513   .0148423     6.62   0.000     .0691609    .1273416
       _cons |

类似地，可以对此模型进行$Probit$估计

In [19]:
stata.run('probit work age married children education, nolog')


Probit regression                                       Number of obs =  2,000
                                                        LR chi2(4)    = 478.32
                                                        Prob > chi2   = 0.0000
Log likelihood = -1027.0616                             Pseudo R2     = 0.1889

------------------------------------------------------------------------------
        work | Coefficient  Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
         age |   .0347211   .0042293     8.21   0.000     .0264318    .0430105
     married |   .4308575    .074208     5.81   0.000     .2854125    .5763025
    children |   .4473249   .0287417    15.56   0.000     .3909922    .5036576
   education |   .0583645   .0109742     5.32   0.000     .0368555    .0798735
       _cons |  -2.467365   .1925635   -12.81   0.000    -2.844782   -2.089948
--------------------------------------------------

下面考察$Probit$模型的边际效应及预测准确度

In [22]:
stata.run(
    """
    margins,dydx(*)
    estat clas
    """
)


. 
.     margins,dydx(*)

Average marginal effects                                 Number of obs = 2,000
Model VCE: OIM

Expression: Pr(work), predict()
dy/dx wrt:  age married children education

------------------------------------------------------------------------------
             |            Delta-method
             |      dy/dx   std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
         age |   .0100768   .0011647     8.65   0.000     .0077941    .0123595
     married |   .1250441   .0210541     5.94   0.000     .0837788    .1663094
    children |   .1298233   .0068418    18.98   0.000     .1164137    .1432329
   education |   .0169386   .0031183     5.43   0.000     .0108269    .0230504
------------------------------------------------------------------------------

.     estat clas

Probit model for work

              -------- True --------
Classified |         D            ~D  |      Total
-

从以上各表可以看出，$Logit$模型的边际效应、准$R^2$以及正确预测比率与$Probit$模型几乎完全相同，故可视为基本等价（两者的估计系数虽有差距，但估计系数没有可比性）

在异方差情况下进行$Probit$估计的$Stata$命令为

In [23]:
stata.run('hetprob work age married children education, het(age married children education) nolog')


Heteroskedastic probit model                    Number of obs     =      2,000
                                                Zero outcomes     =        657
                                                Nonzero outcomes  =      1,343

                                                Wald chi2(4)      =      12.80
Log likelihood =  -1026.19                      Prob > chi2       =     0.0123

------------------------------------------------------------------------------
        work | Coefficient  Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
work         |
         age |   .0356192   .0132539     2.69   0.007      .009642    .0615964
     married |   .5068326   .1629512     3.11   0.002     .1874542     .826211
    children |   .5013477   .1510223     3.32   0.001     .2053494     .797346
   education |   .0766883   .0355709     2.16   0.031     .0069706     .146406
       _cons |  -2.799595   .98687

上表的中间部分为原方程（$work$）的估计方程，而下部为对方差方程（$ln\sigma^2$）的估计结果。上表最后一行显示，似然比检验的$p$值为$0.78$,故可以接受"同方差"的原假设

作为稀有事件的实例，陈强（2014）研究了中原王朝被游牧民族征服的决定因素。以每十年作为观测单位建立时间序列数据，从公元前221年秦朝建立至1911年清朝灭亡，共有213个观测值。其中，中原王朝被征服的现象仅发生了7次。

In [25]:
stata.run('use nomadic_conquest.dta, replace')
%stata sum conquered diff age wall drought1


    Variable |        Obs        Mean    Std. dev.       Min        Max
-------------+---------------------------------------------------------
   conquered |        213    .0328638    .1787001          0          1
        diff |        213    15.99061    55.10522       -116        248
         age |        213    9.873239    7.502697          1         29
        wall |        213    .6619718    .4741525          0          1
    drought1 |        212     .440566    .2937834          0          1


由上表可知，"conquered=1"的发生概率仅为$3.29$，或可视为稀有事件。首先，进行普通的$Logit$回归

In [26]:
stata.run('logit conquered diff age wall drought1, r nolog')


Logistic regression                                     Number of obs =    212
                                                        Wald chi2(4)  =  33.12
                                                        Prob > chi2   = 0.0000
Log pseudolikelihood = -21.437566                       Pseudo R2     = 0.3030

------------------------------------------------------------------------------
             |               Robust
   conquered | Coefficient  std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
        diff |   .0386435   .0091584     4.22   0.000     .0206934    .0565936
         age |  -.2634332   .0962616    -2.74   0.006    -.4521025    -.074764
        wall |  -2.486576    1.28163    -1.94   0.052    -4.998525    .0253721
    drought1 |   3.915505    1.37519     2.85   0.004     1.220183    6.610827
       _cons |  -3.361682   1.038785    -3.24   0.001    -5.397663   -1.325702
--------------

上表显示，中原王朝越早于游牧政权建立（根据王朝周期假说，则中原王朝相对较弱）、干旱越严重，则中原王朝被征服的概率越高。另外，在控制相对年龄（$diff$）的情况下，中原王朝的绝对年龄（$age$）越长，被征服的概率反而降低。长城的保护作用也几乎在$5\%$上水平上显著（$p$值为5.2%）。为了便于比较不同模型，下面计算平均边际效应

In [28]:
stata.run('margins, dydx(*)')


Average marginal effects                                   Number of obs = 212
Model VCE: Robust

Expression: Pr(conquered), predict()
dy/dx wrt:  diff age wall drought1

------------------------------------------------------------------------------
             |            Delta-method
             |      dy/dx   std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
        diff |   .0010269   .0004282     2.40   0.016     .0001877    .0018661
         age |  -.0070005   .0040678    -1.72   0.085    -.0149732    .0009722
        wall |  -.0660784   .0330503    -2.00   0.046    -.1308557    -.001301
    drought1 |   .1040508   .0459542     2.26   0.024     .0139821    .1941194
------------------------------------------------------------------------------


下面使用$King\ and\ Zeng(2001a,b)$的方法对$Logit$模型进行偏差修正

In [31]:
stata.run('relogit conquered diff age wall drought1')

(1 missing value generated)


Corrected logit estimates                             Number of obs =      212

------------------------------------------------------------------------------
             |               Robust
   conquered | Coefficient  std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
        diff |   .0311768   .0089471     3.48   0.000     .0136407    .0487129
         age |  -.2010725   .0940423    -2.14   0.033    -.3853921   -.0167529
        wall |  -2.137077   1.252087    -1.71   0.088    -4.591123    .3169691
    drought1 |   3.432347   1.343493     2.55   0.011     .7991486    6.065545
       _cons |  -3.041812   1.014843    -3.00   0.003    -5.030869   -1.052755
------------------------------------------------------------------------------


从上表可以看出，进行偏差修正后，$Logit$模型的估计系数有些变化，而标准误差均有所下降，故变量的显著性基本没变。命令$relogit$并不提供边际效应的计算。下面估计补对数-对数模型

In [32]:
stata.run('cloglog conquered diff age wall drought1, r nolog')


Complementary log-log regression                Number of obs     =        212
                                                Zero outcomes     =        205
                                                Nonzero outcomes  =          7

                                                Wald chi2(4)      =      43.06
Log pseudolikelihood = -21.549974               Prob > chi2       =     0.0000

------------------------------------------------------------------------------
             |               Robust
   conquered | Coefficient  std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
        diff |   .0342854   .0080802     4.24   0.000     .0184485    .0501222
         age |  -.2257135   .0984111    -2.29   0.022    -.4185957   -.0328314
        wall |   -2.28261   1.136038    -2.01   0.045    -4.509203    -.056016
    drought1 |   3.508806   1.314683     2.67   0.008     .9320757    6.085537
       _cons 

从上表可知，使用补对数-对数也并不改变系数的显著性

In [33]:
stata.run('margins, dydx(*)')


Average marginal effects                                   Number of obs = 212
Model VCE: Robust

Expression: Pr(conquered), predict()
dy/dx wrt:  diff age wall drought1

------------------------------------------------------------------------------
             |            Delta-method
             |      dy/dx   std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
        diff |   .0009932    .000422     2.35   0.019     .0001661    .0018204
         age |  -.0065388   .0040806    -1.60   0.109    -.0145366    .0014591
        wall |  -.0661257   .0342384    -1.93   0.053    -.1332318    .0009804
    drought1 |   .1016478   .0478811     2.12   0.034     .0078025    .1954931
------------------------------------------------------------------------------


从上表可知，使用补对数-对数模型估计的边际效应与$Logit$模型十分接近。这说明，在此例中发生频率$3.29\%$的事件可能还不够稀有，故稀有事件偏差并不明显

对于二值选择模型，有时会遇到解释变量为内生变量的清醒。比如，是否买保险取决于收入，但收入或许为内生变量，因为可能存在遗漏变量同时影响收入与买保险的决定。此时，由于有扰动项与内生解释变量相关，使用通常的$Probit$或者$Logit$模型将得不到一致估计。

下面以数据集$mus14data.dta$为例。首先进行一般的$Probit$回归

In [38]:
stata.run('use mus14data.dta, replace')

In [37]:
stata.run('probit ins linc female age age2 educyear married hisp white chronic adl hstatusg, r nolog')


Probit regression                                       Number of obs =  3,197
                                                        Wald chi2(11) = 366.94
                                                        Prob > chi2   = 0.0000
Log pseudolikelihood = -1933.4275                       Pseudo R2     = 0.0946

------------------------------------------------------------------------------
             |               Robust
         ins | Coefficient  std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
        linc |   .3466893   .0402173     8.62   0.000     .2678648    .4255137
      female |  -.0815374   .0508549    -1.60   0.109    -.1812112    .0181364
         age |   .1162879   .1151924     1.01   0.313     -.109485    .3420608
        age2 |  -.0009395   .0008568    -1.10   0.273    -.0026187    .0007397
    educyear |   .0464387   .0089917     5.16   0.000     .0288153    .0640622
     married |

上表显示，$linc$的系数为$0.35$，且在$1\%$水平上显著，即家庭收入越高，越可能购买私人健康保险。但是，怀疑$linc$为内生变量，因为可能存在同时影响$ins$与$linc$的遗漏变量。为此，使用数据集中另一变量$retire$（本人是否退休）作为工具变量。是否退休显然与收入$linc$相关，满足工具变量的相关性；另一方面，假设是否退休不直接影响购买私人健康保险的决策，故满足工具变量的外生性。下面使用此工具变量进行$IV Probit$估计

In [43]:
stata.run('ivprobit ins female age age2 educyear married hisp white chronic adl hstatusg (linc=retire), nolog')


Probit model with endogenous regressors                 Number of obs =  3,197
                                                        Wald chi2(11) = 389.97
Log likelihood = -5408.3856                             Prob > chi2   = 0.0000

------------------------------------------------------------------------------
             | Coefficient  Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
        linc |  -.6198952   .3585575    -1.73   0.084    -1.322655    .0828646
      female |  -.1424774   .0480326    -2.97   0.003    -.2366195   -.0483352
         age |    .298702   .1196303     2.50   0.013      .064231     .533173
        age2 |   -.002233   .0008763    -2.55   0.011    -.0039504   -.0005156
    educyear |   .1185119   .0209574     5.65   0.000     .0774361    .1595878
     married |   .7570917   .2152097     3.52   0.000     .3352885    1.178895
        hisp |  -.5103816   .0994102    -5.13   0.

上表底部提供了对外生性原假设 $H_0: \rho = 0$ 的沃尔德检验结果，其 $p$ 值为 $0.0367$，故可在 $5\%$ 的水平上认为 $\text{linc}$ 为内生变量。根据模型估计结果，$u_i$ 与 $v_i$ 的相关系数为 $\hat{\rho} = 0.64$，这表明，未度量的遗漏变量在增加家庭收入的同时，也会提高购买保险的倾向。相关系数高达 $0.64$ 也意味着 ivprobit 的估计结果会与 probit 大不相同。事实上，使用 ivprobit 时，$\text{linc}$ 的估计系数为 $-0.62$，不仅改变了符号，而且在 $10\%$ 水平上显著 ($p$ 值为 $0.083$)。这表明，如果使用一般的 Probit 模型进行估计，由于忽略了 $\text{linc}$ 的内生性，将高估家庭收入对于保险购买倾向的正作用。


有时候需要对两个虚拟变量同时考虑其发生的概率。比如，被解释变量之一是病人是否去看医生，而被解释变量之二是病人是否住院。如果对这两个被解释变量分别进行 Probit 建模，则可能损失效率（但依然为一致估计），因为“看医生”与“住院”这两件事通常是相关的，即两个 Probit 方程的扰动项之间可能存在相关性。具体考察以下模型：

\begin{equation}
\left\{
\begin{aligned}
y_{1}^{*} &= x_{1}' \beta_{1} + \varepsilon_{1} \\
y_{2}^{*} &= x_{2}' \beta_{2} + \varepsilon_{2}
\end{aligned}
\right.
\tag{11.35}
\end{equation}

其中，$y_{1}^{*}$ 与 $y_{2}^{*}$ 为不可观测的潜变量，扰动项 $(\varepsilon_{1}, \varepsilon_{2})$ 服从二维联合正态分布，期望为 0，方差为 1，而相关系数为 $\rho$，即

\begin{equation}
\begin{pmatrix}
\varepsilon_{1} \\
\varepsilon_{2}
\end{pmatrix}
\sim N \left\{
\begin{pmatrix}
0 \\
0
\end{pmatrix},
\begin{bmatrix}
1 & \rho \\
\rho & 1
\end{bmatrix}
\right\}
\tag{11.36}
\end{equation}


以 Pindyck and Rubin (1998) 所提供的数据集 school.dta 为例。该数据集包括了 1973 年美国密西根州特洛伊市（Troy, Michigan）增加房产税以支持公立学校议案的 95 名投票人信息。主要变量包括: vote（是否投票赞成），private（是否有孩子在私立学校上学），logptax（房产税对数），loginc（家庭年收入对数），school（投票人是否教师），years（在特洛伊市居住的年数）。假设同时关心是否送孩子上私立学校以及是否投票赞成，而这两件事可能相关（如果孩子上私立学校，则可能不愿增加房产税以支持公立学校），故使用双变量 Probit 模型进行估计。

In [20]:
stata.run('use school.dta,replace')
data = stata.pdataframe_from_data()
data.head()

,obs,pub12,pub34,pub5,private,years,school,loginc,logptax,vote,logeduc
0,1,0,1,0,0,10,1,9.770001,7.0475,1,7.206195
1,2,0,1,0,0,8,0,10.021000,7.0475,0,7.609900
2,3,1,0,0,0,4,0,10.021000,7.0475,0,8.278470
3,4,0,1,0,0,13,0,9.433500,6.3969,0,6.821488
4,5,0,1,0,0,3,1,10.021000,7.2792,1,7.686149


In [23]:
stata.run('biprobit (private = loginc logptax school) (vote = loginc logptax years school), r nolog')


Seemingly unrelated bivariate probit                    Number of obs =     95
                                                        Wald chi2(7)  = 611.22
Log pseudolikelihood = -84.695294                       Prob > chi2   = 0.0000

------------------------------------------------------------------------------
             |               Robust
             | Coefficient  std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
private      |
      loginc |   .3911055   .5826198     0.67   0.502    -.7508083    1.533019
     logptax |  -.1877972    .758957    -0.25   0.805    -1.675326    1.299731
      school |  -4.889164   .2553132   -19.15   0.000    -5.389568   -4.388759
       _cons |  -3.795514   6.355104    -0.60   0.550    -16.25129    8.660261
-------------+----------------------------------------------------------------
vote         |
      loginc |   1.262054   .4367689     2.89   0.004     .4060

上表显示，送孩子上私立学校（private）与教师身份（school）显著负相关；而投票赞成（vote）与家庭收入（loginc）、教师身份（school）显著正相关，与房地产税（logptax）显著负相关，而与居住年数（years）无关。上表最后一行对原假设“$H_0: \rho = 0$”的沃尔德检验显示，$p$值高达0.40，故可认为$\rho=0$，没必要使用双变量 Probit 模型。

为了比较，下面对变量 vote 进行单独的 Probit 估计。


In [24]:
stata.run('probit vote loginc logptax years school, r nolog')


Probit regression                                       Number of obs =     95
                                                        Wald chi2(4)  =  22.07
                                                        Prob > chi2   = 0.0002
Log pseudolikelihood = -54.693088                       Pseudo R2     = 0.1324

------------------------------------------------------------------------------
             |               Robust
        vote | Coefficient  std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
      loginc |   1.260286   .4369633     2.88   0.004      .403854    2.116719
     logptax |   -1.23849   .5281871    -2.34   0.019    -2.273717   -.2032619
       years |  -.0130624   .0177873    -0.73   0.463    -.0479248    .0217999
      school |    1.72503   .4363312     3.95   0.000     .8698368    2.580224
       _cons |  -3.646993   4.479862    -0.81   0.416    -12.42736    5.133376
--------------

从上表可知，单独的 Probit 估计结果（系数与标准误）与双变量 Probit 很接近，这也印证了没必要使用双变量 Probit。